In [ ]:
!pip install networkx
!pip install networkx matplotlib ipywidgets
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
import networkx as nx
import random
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

class NetworkSimulator:
    def __init__(self):
        self.graph = nx.Graph()

    def create_topology(self, topology_type, num_nodes):
        self.graph.clear()
        if topology_type == 'bus':
            self.graph = nx.path_graph(num_nodes)
        elif topology_type == 'ring':
            self.graph = nx.cycle_graph(num_nodes)
        elif topology_type == 'star':
            self.graph = nx.star_graph(num_nodes - 1)
        elif topology_type == 'mesh':
            self.graph = nx.complete_graph(num_nodes)

        # Assign random weights to edges
        for (u, v) in self.graph.edges():
            self.graph[u][v]['weight'] = random.randint(1, 10)

    def hot_potato_routing(self, source, target):
        path = [source]
        current = source
        while current != target:
            neighbors = list(self.graph.neighbors(current))
            if target in neighbors:
                current = target
            else:
                current = random.choice(neighbors)
            path.append(current)
        return path

    def flooding(self, source, target):
        visited = set()
        queue = [(source, [source])]
        while queue:
            (node, path) = queue.pop(0)
            if node not in visited:
                visited.add(node)
                if node == target:
                    return path
                for neighbor in self.graph.neighbors(node):
                    if neighbor not in visited:
                        queue.append((neighbor, path + [neighbor]))
        return None

    def source_routing(self, source, target):
        return nx.shortest_path(self.graph, source, target, weight='weight')

    def distance_vector_routing(self, source, target):
        return nx.shortest_path(self.graph, source, target, weight='weight')

    def link_state_routing(self, source, target):
        return nx.shortest_path(self.graph, source, target, weight='weight')

    def simulate_routing(self, algorithm, source, target):
        if algorithm == 'hot-potato':
            return self.hot_potato_routing(source, target)
        elif algorithm == 'flooding':
            return self.flooding(source, target)
        elif algorithm == 'source':
            return self.source_routing(source, target)
        elif algorithm in ['distance-vector', 'rip', 'link-state']:
            return self.distance_vector_routing(source, target)
        else:
            raise ValueError("Unknown routing algorithm")

    def animate_packet_routing(self, path):
        pos = nx.spring_layout(self.graph)
        fig, ax = plt.subplots(figsize=(10, 8))

        def update(frame):
            ax.clear()
            nx.draw(self.graph, pos, with_labels=True, node_color='lightblue', node_size=500, font_size=16, font_weight='bold', ax=ax)

            edge_labels = nx.get_edge_attributes(self.graph, 'weight')
            nx.draw_networkx_edge_labels(self.graph, pos, edge_labels=edge_labels, ax=ax)

            # Highlight the path up to the current frame
            path_edges = list(zip(path[:frame+1], path[1:frame+2]))
            nx.draw_networkx_edges(self.graph, pos, edgelist=path_edges, edge_color='r', width=2, ax=ax)

            # Highlight the current node
            if frame < len(path):
                nx.draw_networkx_nodes(self.graph, pos, nodelist=[path[frame]], node_color='r', node_size=700, ax=ax)

            ax.set_title(f"Packet Routing Animation (Step {frame+1})")
            ax.axis('off')

        anim = FuncAnimation(fig, update, frames=len(path), interval=1000, repeat=False)
        plt.close(fig)  # Prevent display of static plot
        return HTML(anim.to_jshtml())

simulator = NetworkSimulator()

def run_simulation(topology, num_nodes, algorithm, source, target):
    simulator.create_topology(topology, num_nodes)
    path = simulator.simulate_routing(algorithm, source, target)
    print(f"Path from {source} to {target} using {algorithm} routing: {path}")
    return simulator.animate_packet_routing(path)

# Interactive widget
interact(run_simulation,
         topology=widgets.Dropdown(options=['bus', 'ring', 'star', 'mesh'], description='Topology:'),
         num_nodes=widgets.IntSlider(min=3, max=10, step=1, value=5, description='Nodes:'),
         algorithm=widgets.Dropdown(options=['hot-potato', 'flooding', 'source', 'distance-vector', 'rip', 'link-state'], description='Algorithm:'),
         source=widgets.IntText(value=0, description='Source:'),
         target=widgets.IntText(value=4, description='Target:'))

interactive(children=(Dropdown(description='Topology:', options=('bus', 'ring', 'star', 'mesh'), value='bus'),…

<function __main__.run_simulation(topology, num_nodes, algorithm, source, target)>